In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import requests
import math
import folium
import folium.plugins


In [4]:
m = folium.Map(location= [39.7392, -104.9903], titles = 'Mapbox Bright', zoom_start=9)

m


In [5]:
df = pd.read_csv('../data/globalterrorism.csv', encoding='ISO-8859-1')

df2 = pd.read_csv('../data/gtd1993.csv', encoding='latin-1')
df2.rename(columns={'ï»¿eventid': 'eventid'}, inplace=True)

df_ = pd.concat([df, df2], axis=0)

df_.rename(columns={'eventid': 'Event_ID', 'iyear': 'Year', 'imonth': 'Month', 'iday': 'Day', 'country_txt':'Country','region_txt':'Region',
                    'latitude': 'Latitude', 'longitude': 'Longitude', 'attacktype1_txt':'AttackType','target1':'Target','nkill':'Fatalities',
                    'nwound':'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_Type','weaptype1_txt':'Weapon_Type',
                    'motive':'Motive'}, inplace=True)

terror = df_[['Event_ID', 'Year', 'Month', 'Day', 'Country', 'Region', 'Latitude', 'Longitude', 'AttackType', 'Target', 'Fatalities', 'Wounded', 
              'Summary', 'Group', 'Target_Type', 'Weapon_Type', 'Motive']]

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
countries = terror[terror['Fatalities'] >= 1].groupby('Country')['Fatalities'].sum().to_frame().reset_index().sort_values('Fatalities', ascending=False)

In [7]:
toposite2 = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson'
worldgeo = json.loads(requests.get(toposite2).text)
m = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(
    geo_data=worldgeo,
    name='choropleth',
    data=countries,
    columns=['Country', 'Fatalities'],  # from my df
    key_on='feature.properties.name'
#     key_on='feature.properties.iso_a3',
    #fill_color='YlGn',
    #fill_opacity=0.7,
    #line_opacity=0.2,
    #legend_name='Something Rate (%)'
).add_to(m)
folium.LayerControl().add_to(m)
m

In [8]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
world_geo = f'{url}/world-countries.json'
json_data = gpd.read_file(f'{url}/world-countries.json')


country_data = terror.groupby(by=['Country'], 
                                 as_index=False).count().sort_values(by='Event_ID', ascending=False).iloc[:, :2]
kill_data = terror.groupby(by=['Country'], as_index=False).sum().sort_values(by='Event_ID', ascending=False).loc[:, ['Country', 'Fatalities']]
# kill_data.head(5)


temp_global = json_data.merge(kill_data, left_on='name', right_on='Country', how='left')
# global_data = temp_global.merge(kill_data, left_on='name', right_on='Country', how='left')

temp_global.head(5)
# print(global_data.head(5))
countries = terror[terror['Fatalities'] >= 1].groupby('Country')['Fatalities'].sum().to_frame().reset_index().sort_values('Fatalities', ascending=False)


# toposite2 = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson'
# worldgeo = json.loads(requests.get(toposite2).text)
m = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(
    geo_data=json_data,
    name='choropleth',
    data=countries,
    columns=['Country', 'Fatalities'],  # from my df
    key_on='feature.properties.name'
#     key_on='feature.properties.iso_a3',
    #fill_color='YlGn',
    #fill_opacity=0.7,
    #line_opacity=0.2,
    #legend_name='Something Rate (%)'
).add_to(m)
folium.LayerControl().add_to(m)
m

NameError: name 'gpd' is not defined

In [9]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime
import geopandas as gpd
from branca.colormap import LinearColormap
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

In [10]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
world_geo = f'{url}/world-countries.json'
json_data = gpd.read_file(f'{url}/world-countries.json')

In [11]:
# terror = df_[['Event_ID', 'Year', 'Month', 'Day', 'Country', 'Region', 'Latitude', 'Longitude', 'AttackType', 'Target', 'Fatalities', 'Wounded', 
#               'Summary', 'Group', 'Target_Type', 'Weapon_Type', 'Motive']]



country_data = terror.groupby(by=['Country'], 
                                 as_index=False).count().sort_values(by='Event_ID', ascending=False).iloc[:, :2]
kill_data = terror.groupby(by=['Country'], 
                                 as_index=False).sum().sort_values(by='Event_ID', 
                                                                   ascending=False).loc[:, ['Country', 'Fatalities']]
temp_global = json_data.merge(country_data, left_on='name', right_on='Country', how='left')
global_data = temp_global.merge(kill_data, left_on='name', right_on='Country', how='left')

m = folium.Map(
    location=[0, 0], 
    zoom_start=1.50,
    tiles='openstreetmap'
)

folium.Choropleth(
    geo_data=json_data,
    name='Ataques Terroristas',
    data=kill_data,
    columns=['Country', 'Event_ID'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    nan_fill_opacity=0.9,
    legend_name='Terrorism Recorded 1970 - 2017',
    popup_function='Teste'
).add_to(m)

Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)

folium.GeoJson(
    global_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(fields=['name', 'Event_ID', 'Fatalities'],
                                  aliases=['Country:', 'Incidents:', 'Victims'],
                                  labels=True,
                                  sticky=True)
).add_to(m)

# m.save('terrorism_incidents.html')
m

KeyError: 'Event_ID'

In [12]:
countries = terror[terror['Fatalities'] >= 1].groupby('Country')['Fatalities'].sum().to_frame().reset_index().sort_values('Fatalities', ascending=False)[:50]
countries['Country'].replace({'United States': 'United States of America'}, inplace=True)

countries.head(50)

,Country,Fatalities
77,Iraq,78591.0
0,Afghanistan,39384.0
123,Pakistan,23849.0
119,Nigeria,22682.0
73,India,19865.8
154,Sri Lanka,16586.0
161,Syria,15229.0
31,Colombia,14832.0
128,Peru,12901.0
48,El Salvador,12053.0


In [13]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime
import geopandas as gpd
from branca.colormap import LinearColormap
import os
import json
import matplotlib.pyplot as plt

import seaborn as sns



In [14]:



toposite2 = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson'
worldgeo = json.loads(requests.get(toposite2).text)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
world_geo = f'{url}/world-countries.json'
json_data = gpd.read_file(f'{url}/world-countries.json')
worldgeo_ = json.loads(requests.get(world_geo).text)

countries = terror[terror['Fatalities'] >= 1].groupby('Country')['Fatalities'].sum().to_frame().reset_index().sort_values('Fatalities', ascending=False)[:20]
countries['Country'].replace({'United States': 'United States of America'},inplace=True)

# countries.head()
worldgeo_


m = folium.Map(
    location=[0, 0], 
    zoom_start=1.50,
    tiles='openstreetmap'
)

folium.Choropleth(
    geo_data=worldgeo_,
    name='Ataques Terroristas',
    data=countries,
    columns=['Country', 'Fatalities'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    nan_fill_opacity=0.9,
    legend_name='Terrorism Recorded 1970 - 2017',
    popup_function='Teste'
).add_to(m)

Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)

m


In [94]:





toposite2 = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson'
worldgeo = json.loads(requests.get(toposite2).text)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
world_geo = f'{url}/world-countries.json'
json_data = gpd.read_file(f'{url}/world-countries.json')


worldgeo_= json.loads(requests.get(world_geo).text)
worldgeo_


countries = terror[terror['Fatalities'] >= 1].groupby('Country')['Fatalities'].sum().to_frame().reset_index().sort_values('Fatalities', ascending=False)

# countries.head()
# worldgeo_


m = folium.Map(
    location=[0, 0], 
    zoom_start=1.50,
    tiles='openstreetmap'
)

folium.Choropleth(
    geo_data=worldgeo_,
    name='Ataques Terroristas',
    data=countries,
    columns=['Country', 'Fatalities'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    nan_fill_opacity=0.9,
    legend_name='Terrorism Recorded 1970 - 2017',
    popup_function='Teste'
).add_to(m)
m

In [100]:
def invert_dictionary(d):
    '''
    Given a dictionary d, return a new dictionary with d's values
    as keys and the value for a given key being
    the set of d's keys which shared the same value.

    Parameters
    ----------
    d: dict

    Returns
    -------
    dict
        A dictionary of sets of input keys indexing the same input values
        indexed by the input values.


    >>>invert_dictionary({'a': 2, 'b': 4, 'c': 2})
    {2: {'a', 'c'}, 4: {'b'}}
    '''
    # new_dic = {v: k for k, v in d.items()}
    
    # return new_dic
    
    dictresult= {}
    for k, v in d.items():
        if v not in dictresult:
            dictresult[v] = {k}
        else:
            dictresult[v].add(k)
            
    return dictresult

print(invert_dictionary({'a': 2, 'b': 4, 'c': 2}))

{2: {'a', 'c'}, 4: {'b'}}
